<a href="https://colab.research.google.com/github/HemaKoppanathi-03/book-recommendation/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip -o book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Filter users with at least 200 ratings
user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index
df_ratings = df_ratings[df_ratings['user'].isin(active_users)]

# Filter books with at least 100 ratings
book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index
df_ratings = df_ratings[df_ratings['isbn'].isin(popular_books)]
df = pd.merge(df_ratings, df_books, on='isbn')
df = df[['user', 'title', 'rating']]
book_user_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)
book_user_sparse = csr_matrix(book_user_matrix.values)
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)


In [ ]:
def get_recommends(book=""):
    idx = book_user_matrix.index.get_loc(book)
    distances, indices = model.kneighbors(book_user_matrix.iloc[idx, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(distances[0])):  # Skip the first (input book itself)
        title = book_user_matrix.index[indices[0][i]]
        distance = float(distances[0][i])
        recommended_books.append([title, distance])

    return [book, recommended_books]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False

    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]

    for i in range(4):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False

    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()
